In [2]:
import polars as pl
import pandas as pd

## Abro sitios y reviews

In [3]:
pl_sitios = pl.read_parquet('parquet/sitios_FL.parquet')
pl_reviews = pl.read_parquet('parquet/review_florida.parquet')
print(f'sitios = {pl_sitios.shape}, reviews = {pl_reviews.shape}')

sitios = (4127, 30), reviews = (2850000, 9)


In [4]:
pl_reviews.head()

user_id,name,time,rating,text,pics,resp,gmap_id,__index_level_0__
f64,str,i64,i8,str,list[struct[1]],f64,str,i64
1.0147e20,"""Julie A. Gerbe...",1628003250740,1,"""Update: Their ...",null,null,"""0x8893863ea87b...",0
1.1548e20,"""Martin Sheffie...",1595031217005,5,"""He's a knowled...",null,null,"""0x8893863ea87b...",1
1.0181e20,"""Brian Truett""",1522924253567,5,"""Best doctor I'...",null,null,"""0x8893863ea87b...",2
1.0634e20,"""Tina Sun""",1467907819586,1,"""I was told he ...",null,null,"""0x8893863ea87b...",3
1.0088e20,"""James Haynes""",1480683415081,5,"""Takes the time...",null,null,"""0x8893863ea87b...",4


In [5]:
df_reviews = pl_reviews.to_pandas()
df_reviews.head()

,user_id,name,time,rating,text,pics,resp,gmap_id,__index_level_0__
0,1.014719e+20,Julie A. Gerber,1628003250740,1,"Update: Their “reply” to my review amounted to nothing. They never followed up with me. This doctor has left me without medication and not enough time to find a new doctor and get my prescriptions refilled because I came down with COVID. I have been abandoned with no recourse. I will be calling the state to file a complaint and will not drop this. Having to reschedule two to three appointments (appointments were not back to back as I was accused) over the course of two years (with more than twenty four hours notice) is NOT grounds to abandon your patients. I’m still fuming over the male at the front desk who laughed like a hyena when I questioned why I was being dropped. Steer clear of this place. UPDATE: I followed their link and submitted report. I’ll update here if anything comes from it. Forgot to mention, I was told I missed three appointments in a row by the laughing man on staff. It isn’t true. I missed one because I had been sick and needed labs done before my appointment. I had just seen the doctor and was given new meds at my previous appointment. I had an appointment already scheduled for September and while I’m waiting on refills, they instead cancel me and remove me as a patient. Love the doctor but was cancelled as a patient after calling in to reschedule three (only two with this doctor-one was with previous doctor) appointments (they were not consecutive appointments, and I always called in advance so they could fill the spot) so they have now cancelled my next appointment and refuse to fill my meds. No one notified me. So much for all my family going to one doctor. I now get to move myself, my mom, and my husband to another office. Office manager guy repeatedly laughed at me when I called to question what happened. As a nurse, his behavior was rude and unprofessional. I loved this doctor and his nurse. It’s a shame I’m punished for doing what I thought was right….. one cancelled for being extremely sick, one because my son was in ICU, and a third because my mom had surgery and I couldn’t leave her…. But they were notified well in advance. I’m angry.",None,NaN,0x8893863ea87bd5dd:0x9383ebf973e74abb,0
1,1.154772e+20,Martin Sheffield,1595031217005,5,"He's a knowledgeable doctor but the way he runs his office is a mess. I referred a lot of people to them and wish now that I hadn't. For instance, they once called me an hour before an appointment and cancelled. Also, they don't do their referrals correctly and just overall seem to hold their patients in low esteem. Here's an example, I called his office last week with severe back pains and they said it would be 3 weeks before he could see me - that I should go to Urgent Care. I then called my former doctor and he said come on over - we'll get to you. That's the difference and that's what you should look for in a PCP. Revised 07/16/20. I'm revising this review to a ""5"" because, as I said before, he's a knowledgeable doctor - superb actually. I've heard that all the office personnel and nurse practitioner have been changed or reassigned. If so, it would be unfair to him to leave a poor rating when they seemed to be the main problem.",None,NaN,0x8893863ea87bd5dd:0x9383ebf973e74abb,1
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be evseen, he's very understanding to my needs and only wants to help me any way he can, I would recommend this doctor too every one I know.",None,NaN,0x8893863ea87bd5dd:0x9383ebf973e74abb,2
3,1.063444e+20,Tina Sun,1467907819586,1,"I was told he is a good doctor. I was trying to call to make a appointment this morning. First call, they told me the girl taking appointment went to the bank. Second call, i was told to hold- 5 minutes, still not come back, so I hung up. Very disappointed! !!",None,NaN,0x8893863ea87bd5dd:0x9383ebf973e74abb,3
4,1.008751e+20,James Haynes,1480683415081,5,Takes the 

In [6]:
df_reviews['newtime'] = pd.to_datetime(df_reviews['time'],unit='ms')

In [7]:
pl_reviews = pl.from_pandas(df_reviews)
pl_reviews.shape

(2850000, 10)

In [8]:
pl_reviews = pl_reviews.with_columns(pl.col('newtime').cast(pl.Date).alias('date'))

In [9]:
pl_reviews.head()

user_id,name,time,rating,text,pics,resp,gmap_id,__index_level_0__,newtime,date
f64,str,i64,i8,str,list[struct[1]],f64,str,i64,datetime[ns],date
1.0147e20,"""Julie A. Gerbe...",1628003250740,1,"""Update: Their ...",null,null,"""0x8893863ea87b...",0,2021-08-03 15:07:30.740,2021-08-03
1.1548e20,"""Martin Sheffie...",1595031217005,5,"""He's a knowled...",null,null,"""0x8893863ea87b...",1,2020-07-18 00:13:37.005,2020-07-18
1.0181e20,"""Brian Truett""",1522924253567,5,"""Best doctor I'...",null,null,"""0x8893863ea87b...",2,2018-04-05 10:30:53.567,2018-04-05
1.0634e20,"""Tina Sun""",1467907819586,1,"""I was told he ...",null,null,"""0x8893863ea87b...",3,2016-07-07 16:10:19.586,2016-07-07
1.0088e20,"""James Haynes""",1480683415081,5,"""Takes the time...",null,null,"""0x8893863ea87b...",4,2016-12-02 12:56:55.081,2016-12-02


## Creo un dataframe a partir de un diccionario que esta compuesto por las combinaciones del
## nombre del restaurant + relative_results
## Se llama pl_name_review

In [18]:
i = 0
cantidad = pl_sitios.shape[0]
dicc = {'name':[],'gmap_id':[], 'zip':[]}
for i in range(0,cantidad-1):
    if pl_sitios[i,'relative_results']:
        lista = pl_sitios[i,'relative_results'].split(',')
        for elem in lista:
            dicc['name'].append(pl_sitios[i,'name'])
            dicc['gmap_id'].append(elem)
            dicc['zip'].append(pl_sitios[i,'zip'])
    i = i + 1
pl_review_FL = pl.DataFrame(dicc)    
pl_review_FL.shape

(13788, 3)

In [19]:
pl_review_FL.head()

name,gmap_id,zip
str,str,i64
"""Fresh Point Co...","""0x88c2e5593fbd...",33782
"""Fresh Point Co...","""0x88c2e686e0c7...",33782
"""Fresh Point Co...","""0x88c2fbbede90...",33782
"""Fresh Point Co...","""0x88c2f228ed88...",33782
"""Oneyda's Baker...","""0x88dae1997e12...",34102


## Hago un join entre 
## pl_name_review y pl_reviews

In [20]:
pl_review_FL = pl_review_FL.join(pl_reviews.select(['gmap_id','name','rating','text','date']), on='gmap_id', how='left')
pl_review_FL.shape

(298225, 7)

In [21]:
pl_review_FL.head()

name,gmap_id,zip,name_right,rating,text,date
str,str,i64,str,i8,str,date
"""Fresh Point Co...","""0x88c2e5593fbd...",33782,null,null,null,null
"""Fresh Point Co...","""0x88c2e686e0c7...",33782,null,null,null,null
"""Fresh Point Co...","""0x88c2fbbede90...",33782,null,null,null,null
"""Fresh Point Co...","""0x88c2f228ed88...",33782,"""Keith Evans""",5,"""Very unassumin...",2021-06-07
"""Fresh Point Co...","""0x88c2f228ed88...",33782,"""Jesse Nieves""",4,"""This family-st...",2020-11-03


## Elimino duplicados

In [22]:
pl_review_FL = (pl_review_FL.groupby(['name','name_right']).agg([
    pl.col('rating').mean(),
    pl.col('text').max(),
    pl.col('date').max(),
    pl.col('zip').max()
])
 .sort('name')
)
pl_review_FL.shape

(248691, 6)

In [23]:
pl_review_FL.columns

['name', 'name_right', 'rating', 'text', 'date', 'zip']

In [24]:
pl_review_FL.shape

(248691, 6)

In [25]:
pl_review_FL.head()

name,name_right,rating,text,date,zip
str,str,f64,str,date,i64
""".""",null,null,null,null,33431
"""1000 Degrees P...",null,null,null,null,33014
"""33 Below Icecr...","""Veronica Franc...",5.0,"""Went for the S...",2019-06-12,33323
"""33 Below Icecr...","""Sonia Bauer""",5.0,"""Delicious!!!!!...",2018-11-30,33323
"""33 Below Icecr...","""Marc V Nation""",4.0,null,2019-08-15,33323


## Salvo como rest_review_FL(parquet y csv)

In [26]:
pl_review_FL.write_parquet('parquet/review_FL.parquet')
pl_review_FL.write_csv('csv/review_FL.csv')
pl_review_FL.write_parquet('PG_Google_Maps/Ornaldo/review_FL.parquet')
pl_review_FL.write_csv('PG_Google_Maps/Ornaldo/review_FL.csv')